In [23]:
#######################Tensorflow 코드 시작부분
import librosa
import pyaudio #마이크를 사용하기 위한 라이브러리
import wave
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, Model
from sklearn.model_selection import train_test_split
import os

##### 변수 설정 부분 #####
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100 #비트레이트 설정
CHUNK = int(RATE / 10) # 버퍼 사이즈 1초당 44100비트레이트 이므로 100ms단위
RECORD_SECONDS = 5 #녹음할 시간 설정
WAVE_OUTPUT_FILENAME = "output.wav"
DATA_PATH = "./data/"
X_train = []#train_data 저장할 공간
X_test = []
Y_train = []
Y_test = []
tf_classes = 0

def load_wave_generator(path):

    batch_waves = []
    labels = []
    X_data = []
    Y_label = []
    global X_train, X_test, Y_train, Y_test, tf_classes

    folders = os.listdir(path)

    for folder in folders:
        if not os.path.isdir(path):continue #폴더가 아니면 continue
        files = os.listdir(path+"/"+folder)
        print("Foldername :",folder,"-",len(files),"files")
        #폴더 이름과 그 폴더에 속하는 파일 갯수 출력
        for wav in files:
            if not wav.endswith(".wav"):continue
            else:
                print("Filename :",wav)#.wav 파일이 아니면 continue
                y, sr = librosa.load(path+"/"+folder+"/"+wav)
                mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=int(sr*0.01),n_fft=int(sr*0.02)).T

                X_data.extend(mfcc)
                #print("MFCC Size : ", len(mfcc))

                label = [0 for i in range(len(folders))]
                label[tf_classes] = 1

                for i in range(len(mfcc)):
                    Y_label.append(label)
        tf_classes = tf_classes+1
    #end loop
    print("X_data :",np.shape(X_data))
    print("Y_label :",np.shape(Y_label))
    X_train, X_test, Y_train, Y_test = train_test_split(np.array(X_data), np.array(Y_label))

    xy = (X_train, X_test, Y_train, Y_test)
    np.save("./data.npy",xy)

print("Start FIle Reading")
load_wave_generator(DATA_PATH)

print(tf_classes," classes!!")
print("X_train :",np.shape(X_train))
print("Y_train :",np.shape(Y_train))
print("X_test :",np.shape(X_test))
print("Y_test :",np.shape(Y_test))

Start FIle Reading
Foldername : 0 - 20 files
Filename : youinna1.wav
Filename : youinna10.wav
Filename : youinna11.wav
Filename : youinna12.wav
Filename : youinna13.wav
Filename : youinna14.wav
Filename : youinna15.wav
Filename : youinna2.wav
Filename : youinna3.wav
Filename : youinna4.wav
Filename : youinna5.wav
Filename : youinna6.wav
Filename : youinna7.wav
Filename : youinna8.wav
Filename : youinna9.wav
Filename : 유인나16.wav
Filename : 유인나17.wav
Filename : 유인나18.wav
Filename : 유인나19.wav
Filename : 유인나20.wav
Foldername : 1 - 20 files
Filename : baecheolsu1.wav
Filename : baecheolsu10.wav
Filename : baecheolsu11.wav
Filename : baecheolsu12.wav
Filename : baecheolsu13.wav
Filename : baecheolsu14.wav
Filename : baecheolsu2.wav
Filename : baecheolsu3.wav
Filename : baecheolsu4.wav
Filename : baecheolsu5.wav
Filename : baecheolsu6.wav
Filename : baecheolsu7.wav
Filename : baecheolsu8.wav
Filename : baecheolsu9.wav
Filename : 배철수15.wav
Filename : 배철수16.wav
Filename : 배철수17.wav
Filename : 배

In [21]:
print(X_train[0])
print(Y_train[0])

[-349.23873901   78.22611237   81.71148682   86.06054688    7.67272186
  -35.78727722   14.99026585  -20.09018707  -15.3099308   -23.58823776
    8.48728371  -12.3476944    -3.23004007]
[1 0 0 0 0 0]


In [24]:
X_train, X_test, Y_train, Y_test = np.load("./data.npy", allow_pickle=True)
X_train = X_train.astype("float")
X_test = X_test.astype("float")


class MyModel(tf.keras.Model):             #모델 클래스 선언
    
    def __init__(self):
        super(MyModel,self).__init__()
        self.dense1 = layers.Dense(256, activation='relu')
        self.dense2 = layers.Dense(128, activation='relu')
        self.dense3 = layers.Dense(64, activation='relu')
        self.dense4 = layers.Dense(32, activation='relu')
        self.dense5 = layers.Dense(64, activation='relu')
        self.dense6 = layers.Dense(32, activation='relu')
        self.dense7 = layers.Dense(16, activation='relu')
        self.dense8 = layers.Dense(6, activation='softmax')
        
    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        x = self.dense5(x)
        x = self.dense6(x)
        x = self.dense7(x)
        return self.dense8(x)

batch_size = 32
model = MyModel()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=50, batch_size=batch_size)


print('Learning Finished!')


Epoch 1/50
1412/1412 [==============================] - 1s 653us/step - loss: 0.9999 - accuracy: 0.6245
Epoch 2/50
1412/1412 [==============================] - 1s 648us/step - loss: 0.7440 - accuracy: 0.7252
Epoch 3/50
1412/1412 [==============================] - 1s 660us/step - loss: 0.6556 - accuracy: 0.7582
Epoch 4/50
1412/1412 [==============================] - 1s 649us/step - loss: 0.6029 - accuracy: 0.7806
Epoch 5/50
1412/1412 [==============================] - 1s 651us/step - loss: 0.5644 - accuracy: 0.7961
Epoch 6/50
1412/1412 [==============================] - 1s 645us/step - loss: 0.5308 - accuracy: 0.8077
Epoch 7/50
1412/1412 [==============================] - 1s 645us/step - loss: 0.5099 - accuracy: 0.8181
Epoch 8/50
1412/1412 [==============================] - 1s 646us/step - loss: 0.4908 - accuracy: 0.8239
Epoch 9/50
1412/1412 [==============================] - 1s 650us/step - loss: 0.4765 - accuracy: 0.8303
Epoch 10/50
1412/1412 [==============================] - 1s 649u

In [25]:
saver = model.save("MFCC_DNN")    #오류 날떄 안날떄 원인 불명....

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: MFCC_DNN\assets


In [27]:
y, sr = librosa.load("./test_이재은.wav")

X_test = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=int(sr*0.01),n_fft=int(sr*0.02)).T

'''
0 유인나
1 배철수
2 이재은
3 최일구
4 문재인 대통령
'''
label = [0 for i in range(5)]#class가 3개이니까 y_test만드는 과정
label[2] = 1
Y_test = []
for i in range(len(X_test)):
    Y_test.append(label)

print(np.shape(X_test))
print(np.shape(Y_test))

pridict = model.predict(X_test)
rst = [0,0,0,0,0,0]
for i in range(len(pridict)):
    rst[np.argmax(pridict[i])] = rst[np.argmax(pridict[i])] +1
print(rst)
accuracy = (rst[2]/len(pridict))*100
print("accuracy is ", accuracy)

(502, 13)
(502, 5)
[0, 2, 489, 2, 6, 3]
accuracy is  97.41035856573706


In [28]:
y, sr = librosa.load("./test_유인나1.wav")

X_test = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=int(sr*0.01),n_fft=int(sr*0.02)).T

'''
0 유인나
1 배철수
2 이재은
3 최일구
4 문재인 대통령
'''
label = [0 for i in range(5)]#class가 3개이니까 y_test만드는 과정
label[2] = 1
Y_test = []
for i in range(len(X_test)):
    Y_test.append(label)

print(np.shape(X_test))
print(np.shape(Y_test))

pridict = model.predict(X_test)
rst = [0,0,0,0,0,0]
for i in range(len(pridict)):
    rst[np.argmax(pridict[i])] = rst[np.argmax(pridict[i])] +1
print(rst)
accuracy = (rst[0]/len(pridict))*100
print("accuracy is ", accuracy)

(502, 13)
(502, 5)
[403, 22, 39, 6, 28, 4]
accuracy is  80.2788844621514


In [29]:
y, sr = librosa.load("./test_유인나2.wav")

X_test = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=int(sr*0.01),n_fft=int(sr*0.02)).T

'''
0 유인나
1 배철수
2 이재은
3 최일구
4 문재인 대통령
'''
label = [0 for i in range(5)]#class가 3개이니까 y_test만드는 과정
label[2] = 1
Y_test = []
for i in range(len(X_test)):
    Y_test.append(label)

print(np.shape(X_test))
print(np.shape(Y_test))

pridict = model.predict(X_test)
rst = [0,0,0,0,0,0]
for i in range(len(pridict)):
    rst[np.argmax(pridict[i])] = rst[np.argmax(pridict[i])] +1
print(rst)
accuracy = (rst[0]/len(pridict))*100
print("accuracy is ", accuracy)

(502, 13)
(502, 5)
[435, 8, 14, 16, 4, 25]
accuracy is  86.65338645418326


In [30]:
y, sr = librosa.load("./test_배철수.wav")

X_test = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=int(sr*0.01),n_fft=int(sr*0.02)).T

'''
0 유인나
1 배철수
2 이재은
3 최일구
4 문재인 대통령
'''
label = [0 for i in range(5)]#class가 3개이니까 y_test만드는 과정
label[2] = 1
Y_test = []
for i in range(len(X_test)):
    Y_test.append(label)

print(np.shape(X_test))
print(np.shape(Y_test))

pridict = model.predict(X_test)
rst = [0,0,0,0,0,0]
for i in range(len(pridict)):
    rst[np.argmax(pridict[i])] = rst[np.argmax(pridict[i])] +1
print(rst)
accuracy = (rst[1]/len(pridict))*100
print("accuracy is ", accuracy)

(502, 13)
(502, 5)
[6, 419, 31, 22, 13, 11]
accuracy is  83.46613545816733


In [31]:
y, sr = librosa.load("./test_문재인대통령.wav")

X_test = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=int(sr*0.01),n_fft=int(sr*0.02)).T

'''
0 유인나
1 배철수
2 이재은
3 최일구
4 문재인 대통령
'''
label = [0 for i in range(5)]#class가 3개이니까 y_test만드는 과정
label[2] = 1
Y_test = []
for i in range(len(X_test)):
    Y_test.append(label)

print(np.shape(X_test))
print(np.shape(Y_test))

pridict = model.predict(X_test)
rst = [0,0,0,0,0,0]
for i in range(len(pridict)):
    rst[np.argmax(pridict[i])] = rst[np.argmax(pridict[i])] +1
print(rst)
accuracy = (rst[4]/len(pridict))*100
print("accuracy is ", accuracy)

(502, 13)
(502, 5)
[25, 49, 30, 32, 356, 10]
accuracy is  70.91633466135458
